# 랭체인 멀티턴

- LangChain의 RunnableWithMessageHistory를 사용하여 대화기록을 관리하는 챗봇 구현
- 세션 ID별로 대화 기록이 어떻게 분리되어 저장되는지 확인

## 라이브러리 불러오기

In [10]:
from langchain_core.chat_history import InMemoryChatMessageHistory  # 메모리에 대화 기록을 저장하는 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory  # 메시지 기록을 활용해 실행 가능한 래퍼wrapper 클래스
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_core.messages import HumanMessage

## API KEY 불러오기

In [11]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [12]:
load_dotenv()

True

## 대화 기록 함수 정의

In [13]:
model = ChatOpenAI(model="gpt-4o-mini")

# 세션별 대화 기록을 저장할 딕셔너리
store = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(model, get_session_history)

## 첫 번째 대화

In [14]:
config = {"configurable": {"session_id": "abc2"}}  # 세션 ID를 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 김건우야.")],
    config=config,
)

print(response.content)

안녕하세요, 김건우님! 무슨 도움이 필요하신가요?


## 두 번째 대화

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 김건우입니다. 맞나요?


## 세션 테스트

In [16]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

'죄송하지만, 당신의 이름을 알 수 없습니다. 이름이나 기타 개인 정보를 공유하고 싶으시면 말씀해 주세요!'

In [17]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
)

response.content

'우리는 당신의 이름에 대해 이야기했습니다. 당신의 이름은 김건우라고 하셨죠. 다른 주제로 이야기하고 싶으신가요?'

## 스트리밍 응답 출력

In [18]:
config = {"configurable": {"session_id": "abc2"}}
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="")
    # print(r.content, end="|")

김건우라는 이름으로 봐서, 한국 사람일 가능성이 높습니다. 

한국의 문화에 대해 이야기하자면, 다음과 같은 요소들이 있습니다:

1. **음식**: 한국 음식은 다양한 맛과 재료를 가지고 있습니다. 김치, 비빔밥, 불고기 같은 전통음식이 유명합니다.

2. **전통 의상**: 한복은 한국의 전통 의상으로, 주로 명절이나 결혼식 같은 특별한 날에 입습니다.

3. **K-Pop**: 최근 몇 년 동안 한국의 대중 음악은 전 세계적으로 큰 인기를 얻고 있습니다. 방탄소년단(BTS)과 블랙핑크 같은 그룹이 대표적입니다.

4. **명절**: 설날과 추석은 한국에서 중요한 명절로, 가족들이 모여 차례를 지내고 전통 음식을 나눕니다.

5. **예절**: 한국 문화에서는 예의가 매우 중요하게 여겨집니다. 인사할 때의 존칭, 식사 예절 등이 중요합니다.

혹시 다른 주제나 궁금한 점이 있으신가요?